In [1]:
from scipy import stats
import random as rnd

%run BinomialProbability.ipynb

class rule_based_agent():
    def __init__(self, roll, last_bid, bid_history, total_dice, players, Upper = 0.85, Middle_Upper= 0.75, Middle = 0.57,Middle_Lower= 0.40, Lower = 0.18):
        self.last = last_bid
        self.history = bid_history  
        self.roll = roll
        self.total_dice = total_dice
        self.Upper = Upper
        self.Middle_Upper = Middle_Upper
        self.Middle = Middle
        self.Middle_Lower = Middle_Lower
        self.Lower = Lower
        self.Players = players

    def val_to_ace(self, quant = 1):
        bid = [int(math.ceil(self.last[0]/2)), int(1)]
        return bid
    
    def ace_to_val(self, value, quant = 1):
        bid = [int(self.last[0]*2)+quant, int(value)]
        return bid
        
    def increase_val(self, val):
        bid = [int(self.last[0]), int(val)]
        return bid
        
    def increase_quant_and_val(self, val, quant = 1):
        bid = [int(self.last[0]+quant), int(val)]
        return bid
        
    def increase_quant_on_same_val(self, quant = 1):
        bid = [int(self.last[0]+quant), int(self.last[1])]
        return bid
    
    #Check the players roll mode and see if its mentioned before
    def frequency_roll_mode(self, mode):
        count = 0
        for i in self.history:
            if i[1] == 1:
                count +=1
            if mode != 1:
                if i[1] == mode:
                    count += 1

        frequency_roll_mode = count / len(self.history)
        return frequency_roll_mode
    
    #Check previous bids frequency in history - include aces?
    def frequency_last_bid(self):
        count = 0
        for i in self.history:
            if i[1] == self.last[1]:
                count +=1
            if self.last[1]!= 1:
                if i[1] ==1:
                    count +=1
        frequency_last_bid = count / len(self.history)
        return frequency_last_bid
    
    #Check the count of last bid in current players roll
    def last_bid_ratio(self):
    
        count = 0
        for i in self.roll:
            if i == self.last[1]:
                count +=1
                if self.last[1] != 1:
                    if i == 1:
                        count +=1
        last_bid_ratio = count/len(self.roll)
        if self.last[1] == 1:
            last_bid_ratio =last_bid_ratio*2 #Ratio of last bid val in players roll / nº dice
        
        return last_bid_ratio 
    
    #Check how many you have of your mode - i-e strong roll or weak roll
    def mode_ratio(self, mode):
        count = 0
        for i in self.roll:
            if i == mode:
                count +=1
            if mode != 1:
                if i == 1:
                    count +=1
        mode_ratio = count/len(self.roll)
        return mode_ratio
    
    def increase_bid(self, val, quant=1, keep_low = False):
        if val == 1: #current_players roll mode is aces
            if self.last[1]==1: #last bid was on aces too
                bid = self.increase_quant_on_same_val(quant) #+1 aces
            else: #last bid was on another val
                bid = self.val_to_ace(quant) # /2 aces
        else: #roll mode isnt aces
            if self.last[1] ==1: #previous call was on aces
                bid = self.ace_to_val(val, quant) #*2 +1 on val
            else: 
                if keep_low == True:
                #if condition that if there is little dice or that the binom prob is low!!!!
                    if val > self.last[1]:
                        bid = self.increase_val(val)
                    else:
                        bid = self.increase_quant_and_val(val, quant)
                else:
                    bid = self.increase_quant_and_val(val, quant)
        return bid            
    

    def make_bid(self):
        
        dice_vals = (1,2,3,4,5,6)
        player_roll_mode = stats.mode(self.roll)[0]
        if self.last != None:
            #Calculate the probability of success of the previous bid!
            if self.last[1] == 1:
                binom_prob_last = binom_prob(self.total_dice, self.last[0], ace= True)
            else:
                binom_prob_last = binom_prob(self.total_dice, self.last[0], ace=False)
                
        #Not the first bid!!
        if self.history != []: 
            
            #RISK FREE BIDDING!!!
            if binom_prob_last >= self.Upper:
                bluff = rnd.randint(0,1) #50% change of bluffing
                
                if self.total_dice < 15:
                    keep_low = True
                    quant = 1 #+1 if anything no way of keepin lower
                else:
                    keep_low = False
                    quant = rnd.randint(1,2)#jump can be 1 or 2
                
                if bluff == 0: #NO BLUFF
                    bid = self.increase_bid(player_roll_mode, quant, keep_low)
                
                elif bluff == 1: #Bluffff
                    bluff_dice_val = rnd.choice(dice_vals)
                    bid = self.increase_bid(bluff_dice_val, quant, keep_low)
                            
            
            # SAFE BIDDING
            elif self.Upper > binom_prob_last >= self.Middle_Upper: 
                bluff = rnd.randint(1,4) #25% change of bluffing
                
                if bluff != 1: #NO BLUFF
                    bid = self.increase_bid(player_roll_mode)
                    
                else: #Bluffff
                    bluff_dice_val = rnd.choice(dice_vals)
                    bid = self.increase_bid(bluff_dice_val)
            #DANGER ZONE        
            elif self.Middle_Upper > binom_prob_last >= self.Middle_Lower: 
                
                #Check the players roll mode and see if its mentioned before
                #Threshold is that more players than yourself have mentioned it!
                frequency_roll_mode = self.frequency_roll_mode(player_roll_mode)
                threshold_frequency_roll_mode = 1.3/len(self.Players)
                
                #Check previous bids frequency in history - include aces?
                #Threshold is that more players than previous player have mentioned it!
                frequency_last_bid = self.frequency_last_bid()
                threshold_frequency_last_bid = 1.3/len(self.Players)
                
                #Check the count of last bid in current players roll
                # Threshols is #5 dice - 3+, 4dice - 2+, 3dice - 2+, 2dice - 1+, 1dice - 1.
                last_bid_ratio = self.last_bid_ratio()
                threshold_last_bid_ratio = 0.4
                    
                #Check how many you have of your mode - i-e strong roll or weak roll
                # Threshols is #5 dice - 3+, 4dice - 2+, 3dice - 2+, 2dice - 1+, 1dice - 1.
                mode_ratio = self.mode_ratio(player_roll_mode)
                threshold_mode_ratio = 0.4
               
            
                if last_bid_ratio >= threshold_last_bid_ratio: #Players roll is >=40% of this die
                        bid = self.increase_quant_on_same_val()
                    
                else: #Player doesn't have many of last bids die
                    if frequency_last_bid >= threshold_frequency_last_bid: #mentioned previously in the round
                        if binom_prob_last < self.Middle:
                            if mode_ratio >= 0.5:
                                bid = self.increase_bid(player_roll_mode, keep_low = True)
                            else:
                                if frequency_roll_mode>=threshold_frequency_roll_mode:
                                    bid = self.increase_bid(player_roll_mode, keep_low = True)
                                else:
                                    bid = 'Dudo'
                        else:
                            if mode_ratio >= threshold_mode_ratio: #+40% of mode
                                if self.last[1]== player_roll_mode == 1:
                                    bid = 'Calza'
                                else:
                                    bid = self.increase_bid(player_roll_mode, keep_low = True)
                            else:
                                #not sure about this?
                                bid = self.increase_bid(1, keep_low=True)
                    else:
                        if binom_prob_last > self.Middle:
                            bid = self.increase_bid(player_roll_mode,keep_low=True)
                        else:
                            bid = 'Dudo'
                
            elif self.Middle_Lower > binom_prob_last:
                bid = 'Dudo'
                        
        #FIRST BID OF THE ROUND!!
        else: 
            if self.total_dice > 15:
                quant = rnd.randint(2, 3) #bid 2 or 3 for quantity at the star                                            
            else: 
                quant = rnd.randint(1,2) 
            
            #Analyse player's roll to bid
            player_roll_mode = (stats.mode(self.roll)[0])
            
            #At the start, it's less risky to bluff so 1 in 2 times 
            bluff_or_true = rnd.randint(0,1) 
            
            #Bluff
            if bluff_or_true == 0:
                random_number = int(rnd.randint(2,6))#in the first go players cant bid on aces
                bid = [quant, random_number]   
            
            #No bluff
            elif bluff_or_true == 1: 
                if player_roll_mode == 1: 
                    
                    #players can't start bidding on 1, so bid on random number
                    random_number = int(rnd.randint(2,6))
                    bid = [quant, random_number] 
                    
                else: 
                    roll_mode_val = int(player_roll_mode)
                    bid = [quant, roll_mode_val]

        return bid